### 共通の下準備

- 位置情報データ: df

- ある一人のユーザの位置情報データ: df_1user

- OSMから取得した道路ネットワーク: G_proj

In [ ]:
import pandas as pd

df = pd.read_csv("位置情報csvのパス") #パスを記入
df_1user = df[df["userid"]=="調べたいユーザのID"] #ユーザIDを記入

In [ ]:
import osmnx as ox
import geopandas as gpd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point

# -----------------------------
# 1) OSMグラフの準備（中区などポリゴンで取得）
# -----------------------------
place = "Naka Ward, Yokohama, Kanagawa, Japan"
gdf_place = ox.geocode_to_gdf(place)
polygon = gdf_place.loc[0, "geometry"]

G = ox.graph_from_polygon(polygon, network_type="all", simplify=True) #network_typeの部分はwalkやdriveに変更できる
G_proj = ox.project_graph(G)
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G_proj)

# -----------------------------
# 2) ユーザーGPSデータをGeoDataFrame化
# df_1user: lon, lat, recordedat列を持つ DataFrame
# -----------------------------
# recordedat を datetime 型に変換
df_1user["recordedat"] = pd.to_datetime(df_1user["recordedat"])

gdf_user = gpd.GeoDataFrame(
    df_1user,
    geometry=gpd.points_from_xy(df_1user.lon, df_1user.lat),
    crs="EPSG:4326"
)
gdf_user = gdf_user.to_crs(G_proj.graph['crs'])
gdf_user = gdf_user.sort_values("recordedat").reset_index(drop=True)

### ①Nearest-Node Map Matching

= 各GPS測位点を最寄りのノード（交差点）に割り当てるマッチングアルゴリズム

KD-treeを用いて最近傍点探索を高速化

In [ ]:
# -----------------------------
# 3) KD-tree で最寄り交差点（次数>=3）を取得
# -----------------------------
deg_series = pd.Series(dict(G_proj.degree()))
nodes_gdf["degree"] = nodes_gdf.index.map(deg_series)

cand_nodes = nodes_gdf[nodes_gdf["degree"] >= 3].copy()
coords = np.vstack((cand_nodes["x"].values, cand_nodes["y"].values)).T

from scipy.spatial import cKDTree
tree = cKDTree(coords)

pt_coords = np.vstack((gdf_user.geometry.x, gdf_user.geometry.y)).T
dists, idxs = tree.query(pt_coords, k=1)

gdf_user["nearest_node"] = cand_nodes.index.values[idxs]
gdf_user["snap_dist_m"] = dists

# -----------------------------
# 4) 経路復元（shortest path）／連続重複除去
# -----------------------------
node_seq = gdf_user["nearest_node"].tolist()
node_seq = [node_seq[i] for i in range(len(node_seq)) if i == 0 or node_seq[i] != node_seq[i-1]]

full_path_nodes = []
for a, b in zip(node_seq[:-1], node_seq[1:]):
    try:
        sp = nx.shortest_path(G_proj, a, b, weight="length")
    except nx.NetworkXNoPath:
        sp = [a, b]
    if full_path_nodes:
        full_path_nodes.extend(sp[1:])
    else:
        full_path_nodes.extend(sp)

# -----------------------------
# 5) 可視化（GPS vs マップマッチング）
# -----------------------------
gps_x = gdf_user.geometry.x.values
gps_y = gdf_user.geometry.y.values
path_x = [G_proj.nodes[n]['x'] for n in full_path_nodes]
path_y = [G_proj.nodes[n]['y'] for n in full_path_nodes]

fig, ax = plt.subplots(figsize=(10, 10))
ox.plot_graph(G_proj, ax=ax, show=False, close=False, node_size=0, edge_color="lightgray")
ax.plot(gps_x, gps_y, color='blue', linewidth=2, alpha=0.7, label='Original GPS')
ax.plot(path_x, path_y, color='red', linewidth=2, alpha=0.7, label='Matched Path')
ax.legend(fontsize=12)
plt.title("Original GPS vs Map-Matched Path (nearest-nodes matching)")
plt.show()

以下は``ox.distance.nearest_nodes``を用いたバージョンの最寄り交差点探索アルゴリズム

*``ox.distance.nearest_edges`` は内部で KD-tree を使っているので速い!!

In [ ]:
# -----------------------------
# 3) 最近傍ノード探索（ox.distance.nearest_nodes を使用）
# -----------------------------
# GPS座標を配列に変換
x = gdf_user.geometry.x.values
y = gdf_user.geometry.y.values

# 最近傍ノードを一括検索
nearest_nodes = ox.distance.nearest_nodes(G_proj, X=x, Y=y, return_dist=True)

# 戻り値は (node_ids, dists) のタプル
gdf_user["nearest_node"] = nearest_nodes[0]
gdf_user["snap_dist_m"] = nearest_nodes[1]

# -----------------------------
# 4) 経路復元（shortest path）／連続重複除去
# -----------------------------
node_seq = gdf_user["nearest_node"].tolist()
# 連続重複を削除
node_seq = [node_seq[i] for i in range(len(node_seq)) if i == 0 or node_seq[i] != node_seq[i-1]]

full_path_nodes = []
for a, b in zip(node_seq[:-1], node_seq[1:]):
    try:
        sp = nx.shortest_path(G_proj, a, b, weight="length")
    except nx.NetworkXNoPath:
        sp = [a, b]
    if full_path_nodes:
        full_path_nodes.extend(sp[1:])  # 重複を避ける
    else:
        full_path_nodes.extend(sp)

# -----------------------------
# 5) 可視化（GPS vs マップマッチング）
# -----------------------------
gps_x = gdf_user.geometry.x.values
gps_y = gdf_user.geometry.y.values
path_x = [G_proj.nodes[n]['x'] for n in full_path_nodes]
path_y = [G_proj.nodes[n]['y'] for n in full_path_nodes]

fig, ax = plt.subplots(figsize=(10, 10))
ox.plot_graph(G_proj, ax=ax, show=False, close=False, node_size=0, edge_color="lightgray")
ax.plot(gps_x, gps_y, color='blue', linewidth=2, alpha=0.7, label='Original GPS')
ax.plot(path_x, path_y, color='red', linewidth=2, alpha=0.7, label='Matched Path')
ax.legend(fontsize=12)
plt.title("Original GPS vs Map-Matched Path (nearest-nodes matching)")
plt.show()

### ②Nearest-Edge Map Matching

= 各GPS測位点を最寄りのエッジ（道路リンク）に割り当てるマッチングアルゴリズム

In [ ]:
from shapely.ops import nearest_points

# -----------------------------
# 3) 最近傍エッジにスナップ
# -----------------------------
# osmnxのnearest_edgesを使う（内部はKDTreeで効率的）
x = gdf_user.geometry.x.values
y = gdf_user.geometry.y.values
nearest_edges = ox.distance.nearest_edges(G_proj, X=x, Y=y)

gdf_user["nearest_edge"] = nearest_edges

# -----------------------------
# 4) スナップ点を計算
# -----------------------------
snap_points = []
for pt, edge in zip(gdf_user.geometry, gdf_user["nearest_edge"]):
    # edgeは(u, v, key) のタプル
    edge_geom = edges_gdf.loc[edge, "geometry"]
    nearest_pt = nearest_points(pt, edge_geom)[1]
    snap_points.append(nearest_pt)

gdf_user["snap_point"] = snap_points

# -----------------------------
# 5) 可視化
# -----------------------------
gps_x = gdf_user.geometry.x.values
gps_y = gdf_user.geometry.y.values
snap_x = [p.x for p in gdf_user["snap_point"]]
snap_y = [p.y for p in gdf_user["snap_point"]]

fig, ax = plt.subplots(figsize=(10,10))
ox.plot_graph(G_proj, ax=ax, show=False, close=False, node_size=0, edge_color="lightgray")
ax.plot(gps_x, gps_y, "o-", color="blue", alpha=0.6, label="Original GPS")
ax.plot(snap_x, snap_y, "o-", color="red", alpha=0.6, label="Snapped to Road")
ax.legend(fontsize=12)
plt.title("GPS vs Nearest-Edge Map Matching")
plt.show()

### ③Nearest-Node HMM Matching

= 各GPS測位点を近傍ノード（候補$K$個）にHMMに基づいて尤もらしい経路を求める

In [ ]:
from scipy.spatial import cKDTree

# -----------------------------
# 3) 候補ノードをKD-treeで取得（次数>=3を交差点とする）
# -----------------------------
deg_series = pd.Series(dict(G_proj.degree()))
nodes_gdf["degree"] = nodes_gdf.index.map(deg_series)
cand_nodes = nodes_gdf[nodes_gdf["degree"] >= 3].copy()

coords = np.vstack((cand_nodes["x"].values, cand_nodes["y"].values)).T
tree = cKDTree(coords)

# 各GPS点の候補ノードをk個とる
K = 3
pt_coords = np.vstack((gdf_user.geometry.x, gdf_user.geometry.y)).T
dists, idxs = tree.query(pt_coords, k=K)

# candidates[t] = list of dict( node_id, snap_pt, emis_logp )
candidates = []
for t in range(len(gdf_user)):
    cand_list = []
    for k in range(K):
        node_id = cand_nodes.index.values[idxs[t, k]] if K > 1 else cand_nodes.index.values[idxs[t]]
        node_x = G_proj.nodes[node_id]["x"]
        node_y = G_proj.nodes[node_id]["y"]
        snap_pt = Point(node_x, node_y)
        emis_logp = -dists[t, k]**2 / (2 * 20**2)  # emission確率: 距離20mのガウス
        cand_list.append({"node": node_id, "snap_pt": snap_pt, "emis_logp": emis_logp})
    candidates.append(cand_list)

# -----------------------------
# 4) HMM 遷移確率関数
# -----------------------------
def network_distance_between_points(G, x1, y1, x2, y2):
    n1 = ox.distance.nearest_nodes(G, x1, y1)
    n2 = ox.distance.nearest_nodes(G, x2, y2)
    try:
        return nx.shortest_path_length(G, n1, n2, weight="length")
    except nx.NetworkXNoPath:
        return np.inf

def transition_log_prob(G, prev_snap, curr_snap,
                        straight_dist, delta_t, beta=50.0,
                        vmax=50/3.6, sigma_v=5.0):
    net_d = network_distance_between_points(G,
                                            prev_snap.x, prev_snap.y,
                                            curr_snap.x, curr_snap.y)
    if not np.isfinite(net_d):
        return -np.inf

    # 距離一貫性
    base = -abs(net_d - straight_dist) / beta

    if delta_t <= 0:
        return base

    v_net = net_d / delta_t  # [m/s]
    v_obs = straight_dist / delta_t

    # 最大速度制約
    if v_net > vmax * 2:
        return -np.inf

    # 速度差を正規分布で評価
    speed_term = -0.5 * ((v_net - v_obs)/sigma_v)**2
    return base + speed_term

# -----------------------------
# 5) Viterbi アルゴリズム
# -----------------------------
T = len(gdf_user)
dp = []      # 各時刻のスコア
back = []    # バックポインタ

# 初期化
row0 = [cand["emis_logp"] for cand in candidates[0]]
dp.append(np.array(row0))
back.append(np.full(len(row0), -1, dtype=int))

# 再帰
for t in range(1, T):
    row_prev = dp[-1]
    row_curr = np.full(len(candidates[t]), -np.inf, dtype=float)
    back_curr = np.full(len(candidates[t]), -1, dtype=int)

    sd = gdf_user.geometry.iloc[t].distance(gdf_user.geometry.iloc[t-1])
    dt = (gdf_user.iloc[t]["recordedat"] - gdf_user.iloc[t-1]["recordedat"]).total_seconds()

    for j, cand_j in enumerate(candidates[t]):
        best_score, best_i = -np.inf, -1
        for i, cand_i in enumerate(candidates[t-1]):
            trans = transition_log_prob(G_proj,
                                        cand_i["snap_pt"], cand_j["snap_pt"],
                                        straight_dist=sd, delta_t=dt,
                                        beta=50, vmax=50/3.6, sigma_v=3.0)
            if not np.isfinite(trans):
                continue
            score = row_prev[i] + trans + cand_j["emis_logp"]
            if score > best_score:
                best_score, best_i = score, i
        row_curr[j] = best_score
        back_curr[j] = best_i

    dp.append(row_curr)
    back.append(back_curr)

# 復元
path_idx = []
last_idx = int(np.nanargmax(dp[-1]))
for t in reversed(range(T)):
    path_idx.append(last_idx)
    last_idx = back[t][last_idx]
path_idx = path_idx[::-1]

matched_nodes = [candidates[t][path_idx[t]]["node"] for t in range(T)]

# -----------------------------
# 6) 可視化
# -----------------------------
gps_x = gdf_user.geometry.x.values
gps_y = gdf_user.geometry.y.values
path_x = [G_proj.nodes[n]["x"] for n in matched_nodes]
path_y = [G_proj.nodes[n]["y"] for n in matched_nodes]

fig, ax = plt.subplots(figsize=(10,10))
ox.plot_graph(G_proj, ax=ax, show=False, close=False, node_size=0, edge_color="lightgray")

ax.plot(gps_x, gps_y, color='blue', linewidth=2, alpha=0.7, label='Original GPS')
ax.plot(path_x, path_y, color='red', linewidth=2, alpha=0.7, label='HMM Viterbi Path')

ax.legend(fontsize=12)
plt.title(f"HMM-based Map Matching (Viterbi), K={K}")
plt.show()

### ④Nearest-Edge HMM Matching

= 各GPS測位点を近傍エッジ（候補$K$個）にHMMに基づいて尤もらしい経路を求める

In [ ]:
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
from scipy.spatial import cKDTree
import math
import matplotlib.pyplot as plt

# =============================
# 0) ユーティリティ
# =============================
def bearing(p1, p2):
    # p: (x,y) 平面直交座標系（投影済み）
    dx, dy = (p2[0] - p1[0]), (p2[1] - p1[1])
    th = math.degrees(math.atan2(dx, dy))  # 北=0°, 時計回り
    return (th + 360) % 360

def ang_diff(a, b):
    d = abs((a - b + 180) % 360 - 180)
    return d

def project_point_on_linestring(pt, ls: LineString):
    # pt: shapely Point (projected CRS)
    # 返り値: proj_pt, frac (0~1), dist
    if ls.length == 0:  # 退避
        return ls.coords[0], 0.0, pt.distance(ls)

    # shapely 2.x: line_locate_point/line_interpolate_point
    s = ls.project(pt)  # [m] 0~length
    proj_pt = ls.interpolate(s)
    frac = s / ls.length if ls.length > 0 else 0.0
    dist = pt.distance(ls)
    return proj_pt, frac, dist

def edge_bearing_at_fraction(ls: LineString, frac: float, eps=1e-6):
    # frac 近傍の微小区間の向き（近似）
    L = ls.length
    if L == 0: return 0.0
    s1 = max(0.0, L * (frac - 1e-3))
    s2 = min(L, L * (frac + 1e-3))
    p1 = ls.interpolate(s1)
    p2 = ls.interpolate(s2)
    if p1.equals(p2):  # degenerate
        coords = list(ls.coords)
        if len(coords) >= 2:
            p1 = Point(coords[0]); p2 = Point(coords[-1])
        else:
            return 0.0
    return bearing((p1.x, p1.y), (p2.x, p2.y))

def network_distance_between_projected_points(G, edges_gdf, cand_prev, cand_curr):
    """
    射影点 -> 端点 -> ネットワーク -> 端点 -> 射影点 の合成最短距離
    cand_* は dict:
      { 'edge': (u,v,key), 'proj_pt': Point, 'frac': float }
    """
    (u1, v1, k1) = cand_prev['edge']
    (u2, v2, k2) = cand_curr['edge']
    ls1 = edges_gdf.loc[(u1, v1, k1), 'geometry']
    ls2 = edges_gdf.loc[(u2, v2, k2), 'geometry']

    # 射影点→端点（同一エッジ内の沿道距離）
    L1 = ls1.length
    L2 = ls2.length
    # prev側：projからu1 or v1へ
    d_prev_to_u = cand_prev['frac'] * L1
    d_prev_to_v = (1 - cand_prev['frac']) * L1
    # curr側：u2 or v2からprojへ
    d_u_to_curr = cand_curr['frac'] * L2
    d_v_to_curr = (1 - cand_curr['frac']) * L2

    # 4 通りの組合せを試す
    pairs = [
        (('u','u'), (u1, u2), d_prev_to_u, d_u_to_curr),
        (('u','v'), (u1, v2), d_prev_to_u, d_v_to_curr),
        (('v','u'), (v1, u2), d_prev_to_v, d_u_to_curr),
        (('v','v'), (v1, v2), d_prev_to_v, d_v_to_curr),
    ]

    best = np.inf
    for _, (a, b), d1, d2 in pairs:
        try:
            dn = nx.shortest_path_length(G, a, b, weight='length')
            best = min(best, d1 + dn + d2)
        except nx.NetworkXNoPath:
            continue
    return best

# =============================
# 1) データ・グラフ準備
# =============================
place = "Naka Ward, Yokohama, Kanagawa, Japan"
gdf_place = ox.geocode_to_gdf(place)
polygon = gdf_place.loc[0, "geometry"]

G = ox.graph_from_polygon(polygon, network_type="all", simplify=True)
G_proj = ox.project_graph(G)
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G_proj)

# 速度制限（m/s）: 自由に調整
V_MAX = 50/3.6
SIGMA_V = 3.0     # [m/s] 観測速度との差の標準偏差
SIGMA_POS = 20.0  # [m]   位置誤差
K = 5             # 各時刻の候補数
BETA_DIST = 50.0  # 直線 vs ネット距離のスケール
KAPPA_HD = 20.0   # 方位整合の強さ(大ほど厳しい)

# =============================
# 2) GPS を GeoDataFrame に
# =============================
df_1user["recordedat"] = pd.to_datetime(df_1user["recordedat"])
gdf_user = gpd.GeoDataFrame(
    df_1user,
    geometry=gpd.points_from_xy(df_1user.lon, df_1user.lat),
    crs="EPSG:4326"
).to_crs(G_proj.graph['crs'])
gdf_user = gdf_user.sort_values("recordedat").reset_index(drop=True)

# 方位（観測）を計算（隣接点のベアリング）
obs_heading = []
for i in range(len(gdf_user)):
    if 0 < i < len(gdf_user)-1:
        p1 = (gdf_user.geometry.iloc[i-1].x, gdf_user.geometry.iloc[i-1].y)
        p2 = (gdf_user.geometry.iloc[i+1].x, gdf_user.geometry.iloc[i+1].y)
    elif i == 0 and len(gdf_user) > 1:
        p1 = (gdf_user.geometry.iloc[i].x, gdf_user.geometry.iloc[i].y)
        p2 = (gdf_user.geometry.iloc[i+1].x, gdf_user.geometry.iloc[i+1].y)
    elif i == len(gdf_user)-1 and len(gdf_user) > 1:
        p1 = (gdf_user.geometry.iloc[i-1].x, gdf_user.geometry.iloc[i-1].y)
        p2 = (gdf_user.geometry.iloc[i].x, gdf_user.geometry.iloc[i].y)
    else:
        obs_heading.append(0.0); continue
    obs_heading.append(bearing(p1, p2))
gdf_user["obs_heading"] = obs_heading

# =============================
# 3) 各時刻のエッジ候補 K 個
# =============================
# まずはエッジの代表点（重心）で KDTree を作り粗選抜し、真距離で並べ替え
edge_index = list(edges_gdf.index)  # (u,v,key)
edge_centroids = np.array([[geom.centroid.x, geom.centroid.y] for geom in edges_gdf.geometry])
edge_tree = cKDTree(edge_centroids)

candidates = []  # 時刻 t -> [{edge, proj_pt, frac, emis_logp, edge_bearing}, ...]
for t, pt in enumerate(gdf_user.geometry):
    # 粗 K*3 をとり、真の距離（点-ライン距離）で上位 K を選ぶ
    _, idxs = edge_tree.query([pt.x, pt.y], k=min(K*3, len(edge_centroids)))
    idxs = np.atleast_1d(idxs)

    # 距離で精選
    tmp = []
    for idx in idxs:
        e = edge_index[idx]
        ls = edges_gdf.loc[e, "geometry"]
        proj_pt, frac, dist = project_point_on_linestring(pt, ls)
        tmp.append((dist, e, proj_pt, frac))
    tmp.sort(key=lambda x: x[0])
    top = tmp[:min(K, len(tmp))]

    # emission: 位置ガウス + 方位整合（von Mises ≒ 角度ガウス）
    cand_list = []
    for dist, e, proj_pt, frac in top:
        ls = edges_gdf.loc[e, "geometry"]
        hd_edge = edge_bearing_at_fraction(ls, frac)
        # 位置
        emis_pos = - (dist**2) / (2 * SIGMA_POS**2)
        # 方位
        hd_obs = gdf_user.iloc[t]["obs_heading"]
        dth = ang_diff(hd_obs, hd_edge)
        # 角度をガウス近似（0~180 を扱うため wrap 後の差分）
        emis_hd = - 0.5 * (dth / (180/np.sqrt(KAPPA_HD)))**2  # 強さの調整式
        emis = emis_pos + emis_hd
        cand_list.append({
            "edge": e,
            "proj_pt": proj_pt,
            "frac": frac,
            "emis_logp": emis,
            "edge_bearing": hd_edge
        })
    candidates.append(cand_list)

# =============================
# 4) 遷移確率
# =============================
def transition_log_prob_edge(G, edges_gdf, prev_cand, curr_cand, straight_dist, dt):
    # ネットワーク距離
    net_d = network_distance_between_projected_points(G, edges_gdf, prev_cand, curr_cand)
    if not np.isfinite(net_d):
        return -np.inf

    # 直線-ネット距離整合（指数型）
    base = -abs(net_d - straight_dist) / BETA_DIST

    if dt > 0:
        v_net = net_d / dt
        v_obs = straight_dist / dt
        # 速度上限制約
        if v_net > V_MAX * 2:
            return -np.inf
        speed_term = -0.5 * ((v_net - v_obs) / SIGMA_V)**2
    else:
        speed_term = 0.0

    # ターン・Uターン罰則（共有ノードで逆向きなら軽く罰則）
    (u1, v1, _k1) = prev_cand['edge']
    (u2, v2, _k2) = curr_cand['edge']
    turn_pen = 0.0
    shared = set([u1, v1]).intersection([u2, v2])
    if len(shared) == 1:
        # 方位の折れ角で軽い罰則
        dth = ang_diff(prev_cand['edge_bearing'], curr_cand['edge_bearing'])
        if dth > 135:
            turn_pen -= 1.0  # Uターンに近い
        elif dth > 90:
            turn_pen -= 0.5

    return base + speed_term + turn_pen

# =============================
# 5) Viterbi
# =============================
T = len(gdf_user)
dp, back = [], []

row0 = np.array([c['emis_logp'] for c in candidates[0]], dtype=float)
dp.append(row0)
back.append(np.full(len(row0), -1, dtype=int))

for t in range(1, T):
    row_prev = dp[-1]
    M = len(candidates[t])
    row_curr = np.full(M, -np.inf, dtype=float)
    back_curr = np.full(M, -1, dtype=int)

    pt_t = gdf_user.geometry.iloc[t]
    pt_tm1 = gdf_user.geometry.iloc[t-1]
    straight_dist = pt_t.distance(pt_tm1)
    dt = (gdf_user.iloc[t]["recordedat"] - gdf_user.iloc[t-1]["recordedat"]).total_seconds()

    for j, cj in enumerate(candidates[t]):
        best_s, best_i = -np.inf, -1
        for i, ci in enumerate(candidates[t-1]):
            trans = transition_log_prob_edge(G_proj, edges_gdf, ci, cj, straight_dist, dt)
            if not np.isfinite(trans): 
                continue
            s = row_prev[i] + trans + cj['emis_logp']
            if s > best_s:
                best_s, best_i = s, i
        row_curr[j] = best_s
        back_curr[j] = best_i

    dp.append(row_curr)
    back.append(back_curr)

# 復元
path_idx = []
last_idx = int(np.nanargmax(dp[-1]))
for t in reversed(range(T)):
    path_idx.append(last_idx)
    last_idx = back[t][last_idx]
path_idx = path_idx[::-1]

matched = [candidates[t][path_idx[t]] for t in range(T)]
matched_edges = [m['edge'] for m in matched]
matched_points = [m['proj_pt'] for m in matched]

# =============================
# 6) 経路復元（ノード列）
# =============================
def path_nodes_from_candidates(G, edges_gdf, matched):
    nodes = []
    for k in range(len(matched)-1):
        a = matched[k]; b = matched[k+1]
        (u1, v1, k1) = a['edge']; (u2, v2, k2) = b['edge']
        ls1 = edges_gdf.loc[(u1, v1, k1), 'geometry']
        ls2 = edges_gdf.loc[(u2, v2, k2), 'geometry']
        L1, L2 = ls1.length, ls2.length

        d_prev_to_u = a['frac'] * L1
        d_prev_to_v = (1 - a['frac']) * L1
        d_u_to_curr = b['frac'] * L2
        d_v_to_curr = (1 - b['frac']) * L2

        choices = [
            ((u1, u2), d_prev_to_u + d_u_to_curr),
            ((u1, v2), d_prev_to_u + d_v_to_curr),
            ((v1, u2), d_prev_to_v + d_u_to_curr),
            ((v1, v2), d_prev_to_v + d_v_to_curr),
        ]
        # 中央部（ネットワーク部分）は最短路
        best = None; best_len = np.inf; best_nodes = None
        for (nA, nB), _local in choices:
            try:
                sp = nx.shortest_path(G, nA, nB, weight='length')
                spl = nx.path_weight(G, sp, weight='length')
                if spl < best_len:
                    best_len = spl
                    best_nodes = sp
            except nx.NetworkXNoPath:
                continue
        if best_nodes is None:
            # fallback
            best_nodes = [u1, v1, u2, v2]

        if k == 0:
            nodes.extend(best_nodes)
        else:
            nodes.extend(best_nodes[1:])  # 重複除去
    return nodes

full_path_nodes = path_nodes_from_candidates(G_proj, edges_gdf, matched)

# =============================
# 7) 可視化
# =============================
gps_x = gdf_user.geometry.x.values
gps_y = gdf_user.geometry.y.values
snap_x = [p.x for p in matched_points]
snap_y = [p.y for p in matched_points]
path_x = [G_proj.nodes[n]["x"] for n in full_path_nodes]
path_y = [G_proj.nodes[n]["y"] for n in full_path_nodes]

fig, ax = plt.subplots(figsize=(10,10))
ox.plot_graph(G_proj, ax=ax, show=False, close=False, node_size=0, edge_color="lightgray")
ax.plot(gps_x, gps_y, "o-", alpha=0.5, label="Original GPS")
ax.plot(snap_x, snap_y, "o", alpha=0.9, label="Snapped (edge HMM)")
ax.plot(path_x, path_y, "-", linewidth=2, alpha=0.8, label="Reconstructed path")
ax.legend()
plt.title("Edge-based HMM Map Matching (Viterbi, heading+speed)")
plt.show()